In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import csv
import os
import random
import string
import seaborn as sns
import math
import re
import warnings
import lightgbm as lgb

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from feature_engine import categorical_encoders as ce

from datetime import datetime
from dateutil.parser import parse
from sklearn import metrics
from unidecode import unidecode
from sklearn.metrics import roc_auc_score

from itertools import combinations

warnings.filterwarnings('ignore')
pd.options.display.max_columns = 50
plt.style.use('ggplot')

%matplotlib inline

In [2]:
data_version_input = "01-10-2020"

In [3]:
df_train = pd.read_csv(f"../../data/kalapa/{data_version_input}/train.csv")
df_test = pd.read_csv(f"../../data/kalapa/{data_version_input}/test.csv")

In [4]:

f =  "currentLocationName"
vc1 = sorted(df_train[f].value_counts().index)
print(vc1)
print(len(vc1))
print("#####################")
vc2 = sorted(df_test[f].value_counts().index)
print(vc2)
print(len(vc2))
a = []
for i in vc2:
    if i not in vc1:
        a.append(i)
print(len(a))
print(a)


['A Luoi (1), Binh Tri Thien, Vietnam', 'Abu Dhabi, United Arab Emirates', 'Ajaccio', 'American Falls, Idaho', 'Amherst, Massachusetts', 'An Biên', 'An Duong, Hải Phòng, Vietnam', 'An Giang (4), Nghia Binh, Vietnam', 'An Khê, Gia Lai, Vietnam', 'An Lac, Thái Bình, Vietnam', 'An Lão, Bình Ðịnh, Vietnam', 'An Lê, Thái Bình, Vietnam', 'An Nhơn', 'An Ninh, Long An, Vietnam', 'An Ninh, Quảng Ninh (Quảng Bình)', 'An Phong, Ðồng Tháp, Vietnam', 'An Phu', 'An Phú (thị trấn)', 'An Thuan, Song Be, Vietnam', 'Angiang, An Giang, Vietnam', 'Anh Son', 'Anh Sơn', 'Anini-y, Antique', 'Ap An Thai (1), Tiền Giang, Vietnam', 'Ap Bao Binh, Vietnam', 'Ap Ben Cam, Vietnam', 'Ap Ben Cau, Tây Ninh, Vietnam', 'Ap Binh Duong (1), Vietnam', 'Ap Binh Hung, Tiền Giang, Vietnam', 'Ap Binh Quang, Long An, Vietnam', 'Ap Binh Thuan, Minh Hai, Vietnam', 'Ap Hung Yen, Song Be, Vietnam', 'Ap Lam Phu, Lâm Ðồng, Vietnam', 'Ap Loi Nhon, Tiền Giang, Vietnam', 'Ap My Tho, Tiền Giang, Vietnam', 'Ap Nhon My, An Giang, Vietnam',

In [5]:
df_transform = pd.concat([df_train.iloc[:,2:], df_test.iloc[:,1:]], axis = 0)

In [6]:
data_type = {
    "cat_num":["Field_22","Field_13", "Field_41", "Field_42","Field_50", "Field_51", "Field_53",\
               "Field_73",  "partner0_A", "partner0_B", "partner0_C", "partner0_D", "partner0_E",\
               "partner0_F", "partner0_G", "partner0_H", "partner1_A", "partner1_C", "partner1_G", "partner1_H",\
               "partner2_A", "partner2_C", "partner2_D", "partner2_F", "partner2_H","partner3_A", "partner3_D",\
               "partner3_E", "partner4_L", "partner5_A", "partner5_D", "partner5_G", "Field_3", "Field_38"],
    
    "cat_str" : ["Field_4", "Field_12","Field_36", "Field_54", "Field_62","Field_65","Field_66",\
                 "info_social_sex", "brief", "gioiTinh","currentLocationState", "Field_61", \
                 "currentLocationCountry", 'homeTownState', 'data.basic_info.locale',\
                "homeTownCountry", "Field_55"], 
    
    "date" : ["Field_1", "Field_2", "Field_5", "Field_6", "Field_7", "Field_8", "Field_9", "Field_11",\
             "Field_15", "Field_25", "Field_32", "Field_33", "Field_35", "Field_40","Field_43", "Field_44", \
              "F_startDate", "F_endDate", "E_startDate", "E_endDate", "C_startDate",
             "C_endDate", "G_startDate", "G_endDate", "A_startDate", "A_endDate", "ngaySinh", "Field_34"],
    
    "address" : ["Field_45", "Field_48", "Field_49", "Field_56", 'homeTownName', "currentLocationCity", \
                 "homeTownCity", "diaChi", "currentLocationName"],
    
    "drop" : ["maCv", "Field_18", "Field_68", "Field_46"]
}

In [7]:
lst = data_type["cat_num"] + data_type["cat_str"] + data_type["date"] + data_type["address"] + data_type["drop"]

In [8]:
"""
for f in df_transform.columns:
    vc = df_transform[f].value_counts()
    print(f"{f} : {len(vc)} : {vc.index[:10]}")
"""

'\nfor f in df_transform.columns:\n    vc = df_transform[f].value_counts()\n    print(f"{f} : {len(vc)} : {vc.index[:10]}")\n'

consider transform: "brief" \
consider drop: "Field_56", "maCv"


In [9]:
df_transform = df_transform.drop(data_type["drop"], axis = 1)
df_transform = df_transform.drop(data_type["address"], axis = 1)

In [10]:
def fix38(x):
    if x == 0.0 or x == "0" or x == "0.0":
        return 0
    if x == 1.0 or x == "1":
        return 1
    return np.nan

In [11]:
def fix54(x):
    if x == "1" or x == "01" or x == ' P. Tân Phú':
        return "VN"
    if x in ['IT', 'GB', 'SE', 'TK', 'BD']:
        return np.nan
    return x

In [12]:
def fix55(x):
    if x in ['BC', 'BG', 'CN', 'CO', 'DB', 'EL', 'FO', 'H5', 'KX', 'N7', 'NR', 'SD', 'SJ', 'ST', 'WD']:
        return np.nan
    return x

In [13]:
def fix61(x):
    if x == 'Chất độc hóa học':
        return np.nan
    return x

In [14]:
def fixcurrentLocationState(x):
    if x in ['Ilhas', 'Daegu', 'Auckland Region', 'Basse-Normandie', 'Chongqing', "Provence-Alpes-Côte d'Azur",\
             'Rajshahi Division', 'Hubei', 'Mandalay', 'Missouri', 'Pathum Thani', 'Central Java', 'New Hampshire',\
             'Odisha', 'South Australia', 'Chon Buri', 'Haute-Normandie', 'Kirov Oblast', 'Primorsky Krai', 'Arizona',\
             'Ulsan', 'Kanagawa Prefecture', 'Metro Manila', 'Kachin State', 'Svay Rieng Province', 'Nagano Prefecture',\
             'Shiga Prefecture', 'Oslo', 'Indiana', 'Hunan', 'Bolikhamsai Province', 'Krasnoyarsk Krai',\
             'Distrito Nacional', 'Chiba Prefecture']:
        return np.nan
    return x

In [15]:
def fixcurrentLocationCountry(x):
    if x in ['Macau', 'New Zealand', 'Dominican Republic']:
        return np.nan
    return x

In [16]:
def fixhomeTownState(x):
    if x in ['Hunan', 'Kachin State', 'Distrito Federal', 'Bizerte Governorate', 'Central Java',\
             'Svay Rieng Province', 'Distrito Nacional', 'County Mayo', 'Odisha', 'Lazio', 'Nevada',\
             'Daegu', 'Arizona', 'Equatoria', 'Fukuoka Prefecture', 'Taoyuan City', 'Kandal Province', \
             'State of Mexico', 'Hebei', 'New South Wales', 'Capital Governorate', 'Oregon']:
        return np.nan
    return x

In [17]:
def fixhomeTownCountry(x):
    if x in ['Australia', 'Bahrain', 'Dominican Republic', 'Ireland', 'South Sudan', 'Tunisia']:
        return np.nan
    return x

In [18]:
def fixdata_basic_info_locale(x):
    if x == "_VN":
        return 'vi_VN'
    elif x in ['lv_LV', 'ru_RU']:
        return np.nan
    return x

In [19]:
def calculate_age(born):
    born = datetime.strptime(born,'%Y-%m-%d').date()
    today = datetime.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [20]:
def combine_gender(s):
    x, y = s
    
    if x != x and y != y:
        return "nan"
    
    if x != x:
        return y.lower()
    
    return x.lower()

def process_gender(df):
    df["gender"] = df[["gioiTinh", "info_social_sex"]].apply(combine_gender, axis=1)
    return df

In [21]:
DATE = ["Field_{}".format(i) for i in [5, 6, 7, 8, 9, 11, 15, 25, 32, 33, 35, 40]]
DATETIME = ["Field_{}".format(i) for i in [1, 2, 43, 44]]

In [22]:
def datetime_normalize(s):
    if s != s:
        return np.nan
    
    s = s.split(".")[0]
    if s[-1] == "Z":
        s = s[:-1]
        
    date, time = s.split("T")
    datetime_obj = datetime.strptime(s, "%Y-%m-%dT%H:%M:%S")
    return datetime_obj

def date_normalize(s):
    if s != s:
        return np.nan
    
    try:
        datetime_obj = datetime.strptime(s, "%m/%d/%Y")
    except:
        datetime_obj = datetime.strptime(s, "%Y-%m-%d")
        
    return datetime_obj

In [23]:
def process_datetime_cols(df):
    cat_cols = []
    for col in DATETIME:
        df[col] = df[col].apply(datetime_normalize)
        
    for col in DATE:
        df[col] = df[col].apply(date_normalize)

    cat_cols += DATE + DATETIME
    for col in DATE + DATETIME:
        df[col] = df[col].dt.strftime('%m-%Y')
    
    for cat in ['F', 'E', 'C', 'G', 'A']:
        df[f'{cat}_startDate'] = pd.to_datetime(df[f"{cat}_startDate"], infer_datetime_format=True)
        df[f'{cat}_endDate'] = pd.to_datetime(df[f"{cat}_endDate"], infer_datetime_format=True)
        
        df[f'{cat}_startDate'] = df[f'{cat}_startDate'].dt.strftime('%m-%Y')
        df[f'{cat}_endDate'] = df[f'{cat}_endDate'].dt.strftime('%m-%Y')
        
        cat_cols.append(f'{cat}_startDate')
        cat_cols.append(f'{cat}_endDate')
    
    for col in cat_cols:
        df[col] = df[col]
        
    return df

In [24]:
def str_normalize(s):
    s = str(s).strip().lower()
    s = re.sub(' +', " ", s)
    return s
def process_location(df):
    for col in ["currentLocationLocationId", "homeTownLocationId", "currentLocationLatitude", "currentLocationLongitude", 
                   "homeTownLatitude", "homeTownLongitude"]:
        df[col].replace(0, np.nan, inplace=True)

    df["currentLocationLocationId"] = df["currentLocationLocationId"].apply(str_normalize)
    df["homeTownLocationId"] = df["homeTownLocationId"].apply(str_normalize)

    return df

In [25]:
df_transform["Field_38"] = df_transform["Field_38"].apply(fix38)
df_transform["Field_54"] = df_transform["Field_54"].apply(fix54)
df_transform["Field_55"] = df_transform["Field_55"].apply(fix55)
df_transform["Field_61"] = df_transform["Field_61"].apply(fix61)

df_transform["currentLocationState"] = df_transform["currentLocationState"].apply(fixcurrentLocationState)
df_transform["currentLocationCountry"] = df_transform["currentLocationCountry"].apply(fixcurrentLocationCountry)
df_transform["homeTownState"] = df_transform["homeTownState"].apply(fixhomeTownState)
df_transform["homeTownCountry"] = df_transform["homeTownCountry"].apply(fixhomeTownCountry)
df_transform['data.basic_info.locale'] = df_transform['data.basic_info.locale'].apply(fixdata_basic_info_locale)
df_transform["age"] = df_transform["ngaySinh"].map(lambda x: calculate_age(x) if x is not np.nan else x)
df_transform["null_sum"] = df_transform.isnull().sum(axis=1)

df_transform = process_gender(df_transform)
df_transform = process_datetime_cols(df_transform)

df_transform = process_location(df_transform)

In [26]:
drop = ["gioiTinh", "info_social_sex", "ngaySinh"]
df_transform = df_transform.drop(drop, axis = 1)

In [27]:
categoricals = []
for header in df_transform.columns:
    if str(df_transform[header].dtypes) is "object":
        categoricals.append(header)
for col in categoricals:
    df_transform[col] = df_transform[col].astype(str)
lbl = LabelEncoder()
for col in categoricals:
    
    encoder = ce.CountFrequencyCategoricalEncoder(encoding_method='frequency')
    
    df_transform[col] = df_transform[col].fillna('None')
    # Only take the common values in Train/Test-set
    common_vals = list(set(df_transform[col]))
    
    # Take if vals appeared both 5 times
    common_vals = set(df_transform[col].value_counts()[df_transform[col].value_counts()> 100].index)
    
    # Replace not-common values with "Missing" or np.NaN      
    df_transform.loc[~df_transform[col].isin(common_vals), col] = 'Missing'

    # Implement LE
    lbl.fit(df_transform[col].tolist())
    df_transform[col] = lbl.transform(df_transform[col])


In [28]:
fill_1 = []
for header in df_transform.columns:
    vc = df_transform[header].value_counts()
    if len(vc) < 2:
        fill_1.append(header)
for col in fill_1:
    df_transform[col] = df_transform[col].fillna(-1)

In [29]:
df_transform = df_transform.fillna(df_transform.mean())

In [30]:
import re
df_transform = df_transform.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [31]:
train_dev = pd.concat([df_train.id, df_train.label, df_transform.iloc[:len(df_train),:]], axis = 1)
test = pd.concat([df_test.id, df_transform.iloc[len(df_train):,:]], axis = 1)

In [42]:
test

,id,Field_1,Field_2,Field_3,Field_4,Field_5,Field_6,Field_7,Field_8,Field_9,Field_10,Field_11,Field_12,Field_13,Field_15,Field_19,Field_20,Field_21,Field_22,Field_23,Field_25,Field_27,Field_28,Field_29,Field_32,...,partner1_C,partner1_G,partner1_H,partner2_A,partner2_C,partner2_D,partner2_F,partner2_H,partner3_A,partner3_D,partner3_E,partner4_L,partner5_A,partner5_D,partner5_G,brief,num_of_phone,Field_78,Field_79,Field_80,Field_81,Field_82,age,null_sum,gender
0,53030,25,26,2.000000,1,17,20,24,28,18,1.000000,27,3,1.0,18,0.000000,1.490000e+06,4.50000,1.000000,43.000000,19,0.000000,0.000000,12.00000,18,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15,2.0,20.527708,34.626439,-4.837180,-39.896724,1,25.000000,38,1
1,53031,22,23,2.000000,1,19,22,24,28,20,1.000000,27,3,1.0,34,0.000000,1.043000e+06,4.50000,1.000000,24.000000,21,0.000000,0.000000,12.00000,20,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,16,1.0,20.527708,34.626439,-4.837180,-39.896724,1,27.000000,36,1
2,53032,34,34,1.535958,2,33,36,35,47,34,1.097707,27,3,-1.0,34,0.276163,3.619303e+06,4.48644,0.999451,31.796962,35,215096.068536,9553.550312,6.56507,34,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1.0,20.527708,34.626439,-4.837180,-39.896724,1,31.093663,107,2
3,53033,25,26,2.000000,1,22,25,30,36,23,1.000000,27,3,1.0,23,0.000000,1.490000e+06,4.50000,1.000000,0.000000,24,0.000000,0.000000,12.00000,23,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15,1.0,20.527708,34.626439,-4.837180,-39.896724,1,23.000000,35,0
4,53034,34,34,1.535958,2,33,36,35,47,34,1.097707,27,3,-1.0,34,0.276163,3.619303e+06,4.48644,0.999451,31.796962,35,215096.068536,9553.550312,6.56507,34,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5,1.0,27.000000,34.626439,27.000000,27.000000,1,31.093663,70,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20376,73406,34,34,1.535958,2,33,36,35,47,34,1.097707,27,3,-1.0,34,0.276163,3.619303e+06,4.48644,0.999451,31.796962,35,215096.068536,9553.550312,6.56507,34,...,0.000682,0.098718,0.015845,0.045413,0.040036,0.040687,0.043368,0.042399,0.330844,0.205208,0.001227,0.283992,0.248712,0.000091,0.007286,1,1.0,24.000000,42.644640,-10.722222,-160.000000,3,31.093663,128,2
20377,73407,34,34,1.535958,2,33,36,35,47,34,1.097707,27,3,-1.0,34,0.276163,3.619303e+06,4.48644,0.999451,31.796962,35,215096.068536,9553.550312,6.56507,34,...,0.000682,0.098718,0.015845,0.045413,0.040036,0.040687,0.043368,0.042399,0.330844,0.205208,0.001227,0.283992,0.248712,0.000091,0.007286,3,1.0,44.000000,13.740300,6.562500,0.000000,3,31.093663,128,2
20378,73408,31,31,1.000000,0,30,33,5,45,31,1.000000,27,3,1.0,34,4.136000,6.162640e+06,4.50000,1.000000,172.000000,32,0.000000,0.000000,24.00000,31,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15,1.0,20.527708,34.626439,-4.837180,-39.896724,1,38.000000,26,0
20379,73409,34,34,1.535958,2,33,36,35,47,34,1.097707,27,3,-1.0,34,0.276163,3.619303e+06,4.48644,0.999451,31.796962,35,215096.068536,9553.550312,6.56507,34,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15,1.0,20.527708,34.626439,-4.837180,-39.896724,1,31.093663,78,1


In [33]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'verbose': 1,
    'learning_rate': 0.05,
    'num_leaves': 31,
    'feature_fraction': 0.7,
    'min_data_in_leaf': 200,
    'bagging_fraction': 0.8,
    'bagging_freq': 20,
    'min_hessian': 0.01,
    'feature_fraction_seed': 2,
    'bagging_seed': 3
}

In [34]:
def compute_gini(labels, preds):
    fpr, tpr, thresholds = metrics.roc_curve(labels, preds, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    gini = 2 * auc - 1
    return gini

In [35]:
iter_predicttion = []
for i in range(1):
    pred_stack = []
    kf = KFold(n_splits = 4, shuffle=True)
    fold = kf.split(train_dev)
    for train_index, dev_index in fold:
        train1 = train_dev.iloc[train_index,2:]
        dev1 = train_dev.iloc[dev_index,2:]

        d_train = lgb.Dataset(train1, label = train_dev.loc[train_index]["label"])
        clf = lgb.train(params,d_train, 2500)
        predictions_test = clf.predict(test.iloc[:,1:])
        predictions_dev = clf.predict(dev1)
        pred_stack.append(predictions_test)
        print(compute_gini(train_dev.loc[dev_index]["label"], predictions_dev))
    predictions = np.asarray(pred_stack)
    predictions = np.mean(predictions, axis = 0)

    iter_predicttion.append(predictions)

0.4684237506378164
0.47875438935448256
0.4754351678659501
0.4793030889254779


In [39]:
predictions = np.asarray(iter_predicttion)
predictions = np.mean(predictions, axis = 0)
predictions = list(predictions)

for i in range(len(predictions)):
    if predictions[i] >= 0.5:
        predictions[i] = 1
    else:
        predictions[i] = 0
        
new_label = pd.DataFrame(predictions, columns = ["label"])

new_data_id = []
for i in range(len(new_label)):
    idx = len(df_train) + i
    new_data_id.append(idx)
new_data_id = pd.DataFrame(new_data_id, columns = ["id"])

new_data =  pd.concat([new_data_id, new_label, test.iloc[:,1:]], axis = 1)

In [40]:
data_version = "10-10-2020-b"

In [41]:
os.makedirs(f"../../data/kalapa/{data_version}/", exist_ok=True)
train_dev.to_csv(f"../../data/kalapa/{data_version}/train.csv", index = False)
test.to_csv(f"../../data/kalapa/{data_version}/test.csv", index = False)
new_data.to_csv(f"../../data/kalapa/{data_version}/new_data.csv", index = False)